In [1]:
import pandas as pd
from nba_api.stats import endpoints as nba_endpoints
from nba_api.live.nba import endpoints as nba_live_endpoints
import plotly.graph_objects as go
from tqdm.notebook import tqdm, trange

In [2]:
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 500)

# FT Analysis

In [3]:
seasons = [
    f"{x}-{(x+1)-2000}" for x in range(int(2010),2025)
]

## Players

In [5]:
players = pd.DataFrame()

In [ ]:
for s in tqdm(seasons):
    playerindex = nba_endpoints.playerindex.PlayerIndex(season=s)
    data = playerindex.data_sets[0].get_dict().get("data")
    headers = playerindex.data_sets[0].get_dict().get("headers")
    df = pd.DataFrame(data, columns=headers)
    players = pd.concat(
        [
            players,
            df
        ],
        axis=0
    )

In [ ]:
players.loc[players["PLAYER_LAST_NAME"]=="Tatum"]

## Games

In [ ]:
teamlogs = nba_endpoints.teamgamelogs

games = pd.DataFrame()
for s in tqdm(seasons):
    games = pd.concat(
        [
            games,
            teamlogs.TeamGameLogs(season_nullable=s).get_data_frames()[0]
        ]
    )
game_ids = games["GAME_ID"]

In [ ]:
game_ids = list(set(game_ids))

In [ ]:
len(game_ids)

In [ ]:
games.head()

## All Free Throws

In [ ]:
game = nba_endpoints.playbyplayv3
all_fts = pandas.DataFrame()
for i in tqdm(range(len(game_ids))):
    print(i) if i%100 == 0 else None
    g = game_ids[i]
    try:
        df = pandas.DataFrame(game.PlayByPlayV3(game_id=g).get_dict().get("game").get("actions"))
        dff = df.loc[
            df["subType"].str.startswith("Free Throw")
        ].assign(gameid=g)
        all_fts = pandas.concat([all_fts, dff], axis=0)
    except Exception as e:
        print(f"ERROR: Game {i+1} out of {len(game_ids)} -> {e}")

In [ ]:
all_fts["gameid"].nunique()

In [ ]:
all_fts.assign(
    made=lambda row: np.where(row["description"].str.contains("MISS"), 0, 1)
)

In [ ]:
all_fts = all_fts.assign(
    made=lambda row: np.where(row["description"].str.contains("MISS"), 0, 1)
)

In [ ]:
all_fts.groupby("subType").agg({
    "made": "sum",
    "actionNumber": "count"
}).reset_index().assign(
    ft_perc=lambda row: row["made"]/row["actionNumber"]
)

In [ ]:
all_fts["made"].sum()/all_fts["actionNumber"].count()

In [ ]:
players

In [ ]:
all_fts.columns

In [ ]:
dff = all_fts.merge(
    players[["PERSON_ID", "PLAYER_FIRST_NAME", "PLAYER_LAST_NAME"]],
    left_on="personId",
    right_on="PERSON_ID",
    how="left"
)[[
    "gameid", "personId", "playerNameI", "PLAYER_FIRST_NAME_y", "PLAYER_LAST_NAME_y", "actionNumber", 
    "period", "teamTricode", "subType", "made"
]].rename(columns={"PLAYER_FIRST_NAME_y": "PLAYER_FIRST_NAME", "PLAYER_LAST_NAME_y": "PLAYER_LAST_NAME"})

In [ ]:
dff.assign(
    ft_type=lambda row: np.where(
        row["subType"].str.contains("Technical"), 
        "Technical",
        np.where(
            ~pd.isnull(row["subType"].str.extract("Free Throw ([A-Za-z]+) \d of \d")),
            row["subType"].str.extract("Free Throw ([A-Za-z]+) \d of \d"),
            "Standard"
        )
    )
)

In [ ]:
tech_fts.columns = ["personId", "made", "total"]

In [ ]:
playerindex = nba_endpoints.playerindex.PlayerIndex()
data = playerindex.data_sets[0].get_dict().get("PlayerIndex").get("data")
headers = playerindex.data_sets[0].get_dict().get("PlayerIndex").get("headers")
players = pd.DataFrame(
    data, columns=headers
)

In [ ]:
tech_fts.merge(
    players[["PERSON_ID", "PLAYER_FIRST_NAME", "PLAYER_LAST_NAME"]],
    left_on="personId",
    right_on="PERSON_ID",
    how="left"
).assign(
    tech_ft_perc=lambda row: row["made"]/row["total"]
).sort_values("total", ascending=False).head(100)